In [1]:
import numpy as np
import torch

In [2]:
# Example: define the radial grid (size 140)
r = np.random.rand(140)  # Example radial grid (adjust to your case)

# Assuming 3D arrays for rho, v_r (shape: 140, 110, 128)
rho = np.random.rand(140, 110, 128)  # Example 3D density array
v_r = np.random.rand(140, 110, 128)  # Example 3D radial velocity array

g = 10  # Example gravitational acceleration array (1D, size 140)
p = np.random.rand(140, 110, 128)  # Example pressure array (1D, size 140)

In [3]:
# Broadcast 1D arrays to the shape (140, 110, 128)
r_expanded = np.broadcast_to(r[:, np.newaxis, np.newaxis], (140, 110, 128))  # Broadcasting r

In [4]:
# Compute first derivatives using np.gradient for the radial direction (axis 0 is the radial direction)
d_rho_dr = np.gradient(rho, r, axis=0)  # First derivative of density w.r.t. r (along axis 0)
dv_r_dr = np.gradient(v_r, r, axis=0)  # First derivative of v_r (radial velocity) w.r.t. r (along axis 0)
d_p_dr = np.gradient(p, r, axis=0)  # First derivative of pressure w.r.t. r (1D array)

In [5]:
# Terms in the equation:
term1 = rho * v_r * dv_r_dr  # Convective term: rho * v_r * (dv_r / dr)
term2 = -d_p_dr  # Pressure gradient term: - dp / dr
term3 = rho * g  # Gravitational term: rho * g_r (Broadcast g_r to match rho's shape)

# print(term1.shape, term2.shape, term3.shape, term4.shape, term5.shape, term6.shape, term7.shape)

# Final equation (note the result will have the same shape as rho, v_r, p, etc.)
equation_result = -term1 + term2 + term3

# The result should be the value on the left-hand side of the equation:
print(equation_result.shape)  # Verify the shape of the result (should be the same as the input arrays)


(140, 110, 128)


In [6]:
equation_result[0, :5, :5]

array([[ 7.53674003e+00,  6.37691759e+00, -2.19212056e+00,
         4.84216128e+00,  5.29794931e-01],
       [-5.23200245e+00, -7.50013373e+00, -1.72584741e+00,
         1.77132509e+01,  1.50299685e+00],
       [-1.04813209e+01, -2.64896036e-01, -5.08709692e+00,
         3.16253759e+00,  4.97409829e+00],
       [ 3.63563914e-01,  1.04578656e+01,  1.00565045e+01,
        -3.80476092e+00, -9.45650908e-01],
       [ 2.51756465e+00,  2.27786342e-03,  5.86795356e+00,
         7.75082970e-01,  9.35482619e+00]])

In [7]:
rho_tensor = torch.tensor(rho)
v_r_tensor = torch.tensor(v_r)
p_tensor = torch.tensor(p)
r_tensor = torch.tensor(r)

In [8]:
dr = float(r_tensor[1]-r_tensor[0])

In [9]:
# Compute first derivatives using np.gradient for the radial direction (axis 0 is the radial direction)
d_rho_dr = torch.gradient(rho_tensor, axis=0)[0] / dr  # First derivative of density w.r.t. r (along axis 0)
dv_r_dr = torch.gradient(v_r_tensor, axis=0)[0] / dr    # First derivative of v_r (radial velocity) w.r.t. r (along axis 0)
d_p_dr = torch.gradient(p_tensor, axis=0)[0]  / dr   # First derivative of pressure w.r.t. r (1D array)

In [10]:
# Terms in the equation:
term1 = rho_tensor * v_r_tensor * dv_r_dr  # Convective term: rho * v_r * (dv_r / dr)
term2 = -d_p_dr  # Pressure gradient term: - dp / dr
term3 = rho * g  # Gravitational term: rho * g_r (Broadcast g_r to match rho's shape)

# print(term1.shape, term2.shape, term3.shape, term4.shape, term5.shape, term6.shape, term7.shape)

# Final equation (note the result will have the same shape as rho, v_r, p, etc.)
equation_result = -term1 + term2 + term3

# The result should be the value on the left-hand side of the equation:
print(equation_result.shape)  # Verify the shape of the result (should be the same as the input arrays)


torch.Size([140, 110, 128])


In [11]:
equation_result[0, :5, :5]

tensor([[ 7.5367e+00,  6.3769e+00, -2.1921e+00,  4.8422e+00,  5.2979e-01],
        [-5.2320e+00, -7.5001e+00, -1.7258e+00,  1.7713e+01,  1.5030e+00],
        [-1.0481e+01, -2.6490e-01, -5.0871e+00,  3.1625e+00,  4.9741e+00],
        [ 3.6356e-01,  1.0458e+01,  1.0057e+01, -3.8048e+00, -9.4565e-01],
        [ 2.5176e+00,  2.2779e-03,  5.8680e+00,  7.7508e-01,  9.3548e+00]],
       dtype=torch.float64)